In [1]:
import tensorflow as tf
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
y_true = np.array([[
        [[1, 0, 0], [0, 0, 1], [0, 0, 1]],
        [[0, 0, 1], [0, 1, 0], [0, 0, 1]],
        [[0, 0, 1], [0, 0, 1], [0, 1, 0]]]])

In [3]:
y_pred = np.array([[
        [[0.1, 0.1, 0.8], [0.1, 0.1, 0.8], [0.1, 0.1, 0.8]],
        [[0.1, 0.1, 0.8], [0.1, 0.1, 0.8], [0.1, 0.1, 0.8]],
        [[0.1, 0.1, 0.8], [0.1, 0.1, 0.8], [0.1, 0.1, 0.8]]]])

In [4]:
print(y_true.shape)
print(y_pred.shape)

(1, 3, 3, 3)
(1, 3, 3, 3)


In [5]:
np.argmax(y_true, axis=3)

array([[[0, 2, 2],
        [2, 1, 2],
        [2, 2, 1]]])

In [6]:
np.argmax(y_pred, axis=3)

array([[[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]]])

In [18]:
def three_classes_mean_iou(y_true, y_pred):
    y_pred = tf.constant(y_pred) 
    y_true = tf.constant(y_true)
    num_classes = 3
    score, up_opt = tf.metrics.mean_iou(y_true, y_pred, num_classes)
    K.get_session().run(tf.local_variables_initializer())
    with tf.control_dependencies([up_opt]):
        score = tf.identity(score)
    return score

with tf.Session() as sess:
    init_op = tf.local_variables_initializer()
    sess.run(init_op)
    print(sess.run(three_classes_mean_iou(y_true, y_pred)))

0.0


In [8]:
def intersection_over_union(y_true, y_pred, epsilon=1e-6):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true + y_pred)

    return (2.0 * (intersection + epsilon)/(union + epsilon))

class_names = ["1", "2", "3"]
def average_intersection_over_union(y_true, y_pred, class_names):
    n_preds = y_pred.shape[0]
    print('\nNumber of validation samples IoU evaulated on: {}'.format(n_preds))

    total_iou = 0
    for c in range(len(class_names)):
        iou = intersection_over_union(y_true[:,:,:,c], y_pred[:,:,:,c])
        print('IoU for {} is: {:.3f}'.format(class_names[c], iou))
        total_iou += iou

    print('Average IoU is: {:.5f}'.format(total_iou/len(class_names)))

average_intersection_over_union(y_true,y_pred, class_names)


Number of validation samples IoU evaulated on: 1
IoU for 1 is: 0.105
IoU for 2 is: 0.138
IoU for 3 is: 0.727
Average IoU is: 0.32349


In [14]:
from keras import losses

weights = np.array([0.007, 0.07, 1.0])
weights = tf.constant(weights, dtype=tf.float64)

# categorical_crossentropy from Keras
def loss0(y_true, y_pred):
    y_pred = K.variable(y_pred) 
    y_true = K.variable(y_true)
    return losses.categorical_crossentropy(y_true, y_pred)

# categorical_crossentropy calculated from scratch
def loss1(y_true, y_pred):
    # Scale predictions so that the class probas of each sample sum to 1
    y_pred /= tf.reduce_sum(y_pred, axis=-1, keepdims=True)

    # Clip to prevent NaN's and Inf's
    _epsilon = tf.convert_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1.0 - _epsilon)

    # Do the loss calculation
    loss = y_true * tf.log(y_pred)
    loss = -tf.reduce_sum(loss, axis=-1)
    return loss

# weighted_categorical_crossentropy 
def loss2(y_true, y_pred):    
    # Scale predictions so that the class probas of each sample sum to 1
    y_pred /= tf.reduce_sum(y_pred, axis=-1, keepdims=True)

    # Clip to prevent NaN's and Inf's
    _epsilon = tf.convert_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1.0 - _epsilon)
    
    # Do the loss calculation
    loss = y_true * tf.log(y_pred) * 1.0/weights
    loss = -tf.reduce_sum(loss, axis=-1)
    return loss

# focal_loss
def loss3(y_true, y_pred):
    alpha=0.25
    gamma=2.0
    # Scale predictions so that the class probas of each sample sum to 1
    y_pred /= tf.reduce_sum(y_pred, axis=-1, keepdims=True)
    
    # Clip to prevent NaN's and Inf's
    _epsilon = tf.convert_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1.0 - _epsilon)
    
    # Do the loss calculation
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    loss = alpha * tf.pow(1.0 - pt, gamma) * tf.log(pt)
    loss = -tf.reduce_sum(loss, axis=-1)

    return loss

# weighted focal_loss
def loss4(y_true, y_pred):
    alpha=1.0/weights
    gamma=2.0
    # Scale predictions so that the class probas of each sample sum to 1
    y_pred /= tf.reduce_sum(y_pred, axis=-1, keepdims=True)
    
    # Clip to prevent NaN's and Inf's
    _epsilon = tf.convert_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1.0 - _epsilon)
    
    # Do the loss calculation
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    loss = alpha * tf.pow(1.0 - pt, gamma) * tf.log(pt)
    loss = -tf.reduce_sum(loss, axis=-1)

    return loss

print(K.eval(loss0(y_true, y_pred)))
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    print(sess.run(loss1(y_true, y_pred)))
    print(sess.run(loss2(y_true, y_pred)))
    print(sess.run(loss3(y_true, y_pred)))
    print(sess.run(loss4(y_true, y_pred)))

[[[2.3025851  0.22314353 0.22314353]
  [0.22314353 2.3025851  0.22314353]
  [0.22314353 0.22314353 2.3025851 ]]]
[[[2.30258509 0.22314355 0.22314355]
  [0.22314355 2.30258509 0.22314355]
  [0.22314355 0.22314355 2.30258509]]]
[[[3.28940728e+02 2.23143551e-01 2.23143551e-01]
  [2.23143551e-01 3.28940728e+01 2.23143551e-01]
  [2.23143551e-01 2.23143551e-01 3.28940728e+01]]]
[[[0.72404695 0.00275824 0.00275824]
  [0.00275824 0.72404695 0.00275824]
  [0.00275824 0.00275824 0.72404695]]]
[[[2.67487081e+02 1.74492267e-01 1.74492267e-01]
  [1.74492267e-01 2.78247542e+01 1.74492267e-01]
  [1.74492267e-01 1.74492267e-01 2.78247542e+01]]]
